# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Открыть файл с данными и изучить общую информацию

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv')
data.info()
print(data.head(5))
print(' ')
print('Значения в столбце Дети: ', data['children'].unique())
print(' ')
print('Значения в столбце Возраст:', data['dob_years'].unique())
print(' ')
print('Значения в столбце Образование:', data['education'].unique())
print(' ')
print('Значения в столбце Айди Образования:', data['education_id'].unique())
print(' ')
print('Значения в столбце Семейное положение:', data['family_status'].unique())
print(' ')
print('Значения в столбце Айди Семейного положения:', data['family_status_id'].unique())
print(' ')
print('Значения в столбце Пол:', data['gender'].unique())
print(' ')
print('Значения в столбце Тип занятости:', data['income_type'].unique())
print(' ')
print('Значения в столбце Наличие просрочки:', data['debt'].unique())
print(' ')
print('Значения в столбце Цель кредита:', data['purpose'].unique())
print(' ')
print('Минимум в столбце Стаж:', data['days_employed'].min(), 'максимум: ', data['days_employed'].max())
print(' ')
print('Минимум в столбце Доход:', data['total_income'].min(), 'максимум: ', data['total_income'].max())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
   children  days_employed  dob_years education  education_id  \
0         1   -8437.673028         42    высшее             0   
1         1   -4024.803754         36   среднее             1   
2         0   -5623.422610         33   Среднее             1   
3         3   -4124.747207         32   среднее             1   
4  

**Вывод**

Всего 21525 строк. 
Айдишки уже присвоены типу образования и семейного статуса. Это значит что в этих полях операторы значения выбирали из списка, а в остальных формах заполняли руками, и в этих полях может быть больше всего ошибок из-за человеческого фактора.
Пропуски в стаже и ежемесячном доходе (одинаковое количество, скорее всего взаимосвязаны). 
Столбец Образование: куча дубликатов + разный регистр - причесать. Хотя столбец с его айди должен быть корректным, там 5 значений, наверняка это Начальное, среднее, неоконч. высшее, высшее, ученая степень, надо проверить соответствие этих индексов. 
В столбце Возраст есть нули - что это?
В столбце Дети есть значения "-1" и "20", скорее всего опечатка при наборе "1" и "2", надо проверить.
Столбец семейного положения  - ОК, тоже надо проверить соответствие столбцу с айди.
В столбце Пол есть "XNA" - WTF?
В столбце Тип занятости можно попробовать обобщить категории.
В Цели кредита объединить в общие категории типа Автомобиль, Недвижимость, Образование, Свадьба.
В Стаже есть отрицательные значения и есть завышенные например 1100 лет - проверить.
В столбце Доход вроде ок.

## Шаг 2. Предобработка данных

### Обработка пропусков

Цель: 
- убрать пропуски в столбцах Среднего стажа и Заработка,
- а также посмотреть нули в Возрасте, 
- убрать косяки в количестве детей 
- и графе Пол.

Посчитаем количество строк с пропусками в стаже и средний стаж в выборке.

In [3]:
print(len(data[data['days_employed'].isna()])) #количество строк с пропусками в стаже
print ('Средний стаж: ', data['days_employed'].mean()) #средний стаж



2174
Средний стаж:  63046.49766147338


2174 строки без указания стажа работы. Этот показатель не участвует в расчетах для целей исследования, поэтому можно было бы заменить их на среднее по больнице значение. Но в исходнике есть отрицательные значения и очень большие, средняя из таблицы выходит 63046 дней, то есть 172 года - не годится. 
Проверим гипотезу: отрицательные значения - это трудоустроенные на сегодня люди с активным стажем.
Сначала посмотрим общую разбивку по типу трудоустройства.

In [4]:
data['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

Посмотрим, кто в таблице в принципе без указания стажа.

In [5]:
print (data[data['days_employed'].isna()].head(20))

     children  days_employed  dob_years education  education_id  \
12          0            NaN         65   среднее             1   
26          0            NaN         41   среднее             1   
29          0            NaN         63   среднее             1   
41          0            NaN         50   среднее             1   
55          0            NaN         54   среднее             1   
65          0            NaN         21   среднее             1   
67          0            NaN         52    высшее             0   
72          1            NaN         32    высшее             0   
82          2            NaN         50    высшее             0   
83          0            NaN         52   среднее             1   
90          2            NaN         35    высшее             0   
94          1            NaN         34    высшее             0   
96          0            NaN         44   СРЕДНЕЕ             1   
97          0            NaN         47    высшее             

Связи не наблюдается, тут все подряд, завиимости не наблюдается. 
Потом посчитаем, какие люди в таблице с отрицательным значением стажа и его среднее значение.

In [6]:
print (data[data['days_employed'] <= 0]['income_type'].value_counts()) #посчитаем какие люди в таблице с отрицательным значением стажа
print ('Всего: ', (data[data['days_employed'] <= 0]['income_type'].value_counts()).sum()) #общая сумма
days_employed_positive_mean = data[data['days_employed'] <= 0]['days_employed'].mean() #выведем средний стаж в этой выборке
print ('Среднее: ', '{:.0f}'.format(days_employed_positive_mean), 'дней или ', '{:.2f}'.format(days_employed_positive_mean/365), 'лет')

сотрудник          10014
компаньон           4577
госслужащий         1312
предприниматель        1
в декрете              1
студент                1
Name: income_type, dtype: int64
Всего:  15906
Среднее:  -2353 дней или  -6.45 лет


В принципе, похоже на правду: здесь есть наемники, компаньоны (кто это кстати? учредители юр лиц с видом доходов дивиденды?) и госслужащие, только затесалось по одному человеку из других групп. 
Теперь проверим наоборот: с положительным стажем.

In [7]:
print (data[data['days_employed'] > 0]['income_type'].value_counts()) #выборка с положительным стажем
print ('Всего: ', (data[data['days_employed'] > 0]['income_type'].value_counts()).sum()) #общая сумма
days_employed_negative_mean = data[data['days_employed'] > 0]['days_employed'].mean() #выведем средний стаж в этой выборке
print ('Среднее: ', '{:.0f}'.format(days_employed_negative_mean), 'дней или ', '{:.0f}'.format(days_employed_negative_mean/365), 'лет')

пенсионер      3443
безработный       2
Name: income_type, dtype: int64
Всего:  3445
Среднее:  365004 дней или  1000 лет


Уже тепло... Здесь сидят пенсионеры и два безработных. Среднее значение стажа у них конечно закшкаливает и портит всю статистику, но это уже другая проблема. Посмотрим, у кого пропуски-то

In [8]:
print (data[data['days_employed'].isna()]['income_type'].value_counts()) #выборка с пропусками в стаже
print ('Всего: ', (data[data['days_employed'].isna()]['income_type'].value_counts()).sum())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64
Всего:  2174


Гипотеза подтвердилась, и сразу же можно увидеть, что пропуски в строках распределены равномерно по всем категориям занятых, проверим на сколько: 

In [9]:
data[data['days_employed'].isna()]['income_type'].value_counts() / data[data['days_employed'] > -20000]['income_type'].value_counts() #разделим пропуски на заполненные строки, -20000 это чтобы выделить непустые ячейки, потому что минимум в таблице был -18000 

безработный             NaN
в декрете               NaN
госслужащий        0.112043
компаньон          0.110990
пенсионер          0.119954
предприниматель    1.000000
сотрудник          0.110346
студент                 NaN
Name: income_type, dtype: float64

Бинго. 11% выпало при выгрузке данных у каждой категории. Совпадение? Не думаю...
Теперь можно смело заполнить пропуски, взяв среднее не у всей выборки, а у своей подкатегории.
Но среднее у пенсионеров и безработных в 1000 лет настораживает. Вероятно, это не дни, а часы. Проверим. 

In [11]:
print(data[data['days_employed'] > 0]['days_employed'].min()) #проверим крайние значения в выборке
print(data[data['days_employed'] > 0]['days_employed'].max())

328728.72060451825
401755.40047533


Самое маленькое значение в этой выборке - 328728, если это часы, то получается 13 697 дней либо 37 лет.
для максимального значения 401755 выходит, что это 45 лет. Для пенсионеров такой стаж в принципе соответствует действительности, поэтому разделим положительные значения на 24 часа. А отрицательные переведем в положительные для корректности всех значений. Также потом добавим столбец стажа в годах, чтобы сравнить с возрастом и проверить корректность данных.

In [12]:
def fix_positive_days_employed (days_employed): #пишем функцию к строке, которая делит положительные значения на 24, а остальные не трогает
    if days_employed > 0:
        return days_employed / 24
    if days_employed <= 0:
        return days_employed * -1
    return days_employed
data['days_employed'] = data['days_employed'].apply(fix_positive_days_employed)
print (data['days_employed'].head(10))

0     8437.673028
1     4024.803754
2     5623.422610
3     4124.747207
4    14177.753002
5      926.185831
6     2879.202052
7      152.779569
8     6929.865299
9     2188.756445
Name: days_employed, dtype: float64


Красота. Теперь заполним пропуски, заполнив не общим средним, а средним по каждой категории занятых, чтобы сохранить репрезентативность.

In [13]:
print ('Пропуски до: ', data['days_employed'].isna().sum())

for income_type in data['income_type'].unique():
    average = data.loc[data['income_type'] == income_type, 'days_employed'].mean()
    print(income_type, average)
    data.loc[ (data['days_employed'].isna()) & (data['income_type'] == income_type), 'days_employed'] = average

print ('Пропуски после: ', data['days_employed'].isna().sum())

Пропуски до:  2174
сотрудник 2326.4992159717935
пенсионер 15208.478801869218
компаньон 2111.524398297732
госслужащий 3399.89690169574
безработный 15267.235531008522
предприниматель 520.8480834953765
студент 578.7515535382181
в декрете 3296.7599620220594
Пропуски после:  0


Теперь возьмемся за заработок. Проверим у кого он не проставлен.

In [14]:
#выборка с пропусками в заработке по типу занятости
print (data[data['total_income'].isna()]['income_type'].value_counts()) 
print ('Всего: ', (data[data['total_income'].isna()]['income_type'].value_counts()).sum())

сотрудник          1105
компаньон           508
пенсионер           413
госслужащий         147
предприниматель       1
Name: income_type, dtype: int64
Всего:  2174


Аналогичная выборка как у пропусков со стажем. Можно заполнить их также по средней в категории занятости, но т.к. этот столбец буден напрямую влиять на результаты исследования, сначала на всякий случай проверим их медианы.

In [15]:
for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'total_income'].median()
    print(income_type, '{:.2f}'.format(median))
    

сотрудник 142594.40
пенсионер 118514.49
компаньон 172357.95
госслужащий 150447.94
безработный 131339.75
предприниматель 499163.14
студент 98201.63
в декрете 53829.13


У нас в выборке у предпринимателей отсутствует доход только у одного из двух, у остальных групп выборка в достаточном количестве, поэтому медианное значение вставим в пропуски. 
Но просто так ставить доход в полмиллиона одному человеку было бы слишком поспешно. Посмотрим на этих двух.

In [16]:
print(data.loc[data['income_type'] == 'предприниматель'])

       children  days_employed  dob_years education  education_id  \
5936          0     520.848083         58    высшее             0   
18697         0     520.848083         27    высшее             0   

          family_status  family_status_id gender      income_type  debt  \
5936    женат / замужем                 0      M  предприниматель     0   
18697  гражданский брак                 1      F  предприниматель     0   

        total_income                     purpose  
5936             NaN  покупка жилой недвижимости  
18697  499163.144947       на проведение свадьбы  


В принципе у них ни у кого не было выходов на просрочку, поэтому заполним у них пропуск медианой.

In [16]:
print ('Пропуски до: ', data['total_income'].isna().sum())

#это выделение медианы по одному критерию: типу занятости
for income_type in data['income_type'].unique():
    median = data.loc[data['income_type'] == income_type, 'total_income'].median()
    print(income_type, '{:.2f}'.format(median))
    data.loc[ (data['total_income'].isna()) & (data['income_type'] == income_type), 'total_income'] = median

#здесь отфильтруем по двум: образованию и типу занятости
#medians = (data.groupby(['income_type','education']).agg({'total_income':'median'}).rename(columns = {'total_income':'total_income_medians'})
#data = data.merge(medians, on = ['income_type','education'])
#data.loc[data['total_income'].isna(), 'income_type'] = data.loc[data['total_income'].isna(), 'total_income_medians']   

print ('Пропуски после: ', data['total_income'].isna().sum())

Пропуски до:  2174
сотрудник 142594.40
пенсионер 118514.49
компаньон 172357.95
госслужащий 150447.94
безработный 131339.75
предприниматель 499163.14
студент 98201.63
в декрете 53829.13
Пропуски после:  0


С пропусками разобрались. 
Теперь посмотрим, что за значения XNA в столбце Пол.

In [34]:
print (data[data['gender'] == 'XNA'])

       children  days_employed  dob_years            education  education_id  \
10701         0    2358.600502         24  неоконченное высшее             2   

          family_status  family_status_id gender income_type  debt  \
10701  гражданский брак                 1    XNA   компаньон     0   

        total_income               purpose  
10701  203905.157261  покупка недвижимости  


Непонятно откуда взялся этот XNA всего в одной строчке, наверное какой-то микробаг. Один человек: 24 года, детей нет, в гражданском браке, уже имеет долю в бизнесе, взял кредит на недвижимость и пока без просрочек. Какой же у него/нее пол? Звучит как детектив. Чтобы определить пол, примем самое рациональное и взвешенное решение - кинем монетку. Монетка сказала Male, меняем.

In [35]:
data['gender'] = data['gender'].replace('XNA', 'M')
#сразу себя проверим
print('Значения в столбце Пол:', data['gender'].unique())


Значения в столбце Пол: ['F' 'M']


В столбце Дети есть значения "-1" и "20", посмотрим как много их

In [36]:
print('Число строк со значением Дети в 20: ', len(data[data['children'] == 20]))
print('Число строк со значением Дети в -1: ',len(data[data['children'] == -1]))


Число строк со значением Дети в 20:  76
Число строк со значением Дети в -1:  47


76 людей с 20ю детьми, причем в таблице у всех остальных не больше 5, поэтому это артефакт. Также и с -1, тут скорее всего опечатка оператора при занесении данных в форму. 
Всего 123 строчки, но оставлять их в таком виде не годится, а исключить жалко, поэтому предположим, что 20 это опечатка с 2 (двойка рядом с нулем на цифровой клавиатуре), а -1 это 1. Исправим это.

In [37]:
data['children'] = data['children'].replace(20, 2)
data['children'] = data['children'].replace(-1, 1)

print('Значения в столбце Дети: ', data['children'].unique())

Значения в столбце Дети:  [1 0 3 2 4 5]


Получилось. Теперь посмотрим у кого 0 в графе Возраст

In [38]:
print (data[data['dob_years'] == 0])

       children  days_employed  dob_years education  education_id  \
99            0   14439.234121          0   Среднее             1   
149           0    2664.273168          0   среднее             1   
270           3    1872.663186          0   среднее             1   
578           0   16577.356876          0   среднее             1   
1040          0    1158.029561          0    высшее             0   
...         ...            ...        ...       ...           ...   
19829         0    2326.499216          0   среднее             1   
20462         0   14113.952856          0   среднее             1   
20577         0   13822.552977          0   среднее             1   
21179         2     108.967042          0    высшее             0   
21313         0    1268.487728          0   среднее             1   

               family_status  family_status_id gender income_type  debt  \
99           женат / замужем                 0      F   пенсионер     0   
149                в 

101 строчка, то же самое: заполнить их средним меньшее из зол, чем вообще удалить. Заполним их средним, только не общим, а в соответствующей категории Типа занятости.

In [39]:
print ('Нули до: ', data[data['dob_years'] == 0]['dob_years'].count())

for income_type in data['income_type'].unique():
    average = data.loc[data['income_type'] == income_type, 'dob_years'].mean()
    print(income_type, average)
    data.loc[ (data['dob_years'] == 0) & (data['income_type'] == income_type), 'dob_years'] = average

print ('Пропуски после: ', data[data['dob_years'] == 0]['dob_years'].count())

Нули до:  101
сотрудник 39.82102707077975
пенсионер 59.06301867219917
компаньон 39.69754178957719
госслужащий 40.63673749143249
безработный 38.0
предприниматель 42.5
студент 22.0
в декрете 39.0
Пропуски после:  0


В конце проверим, как встали наши средние значения. Вдруг где-то внезапно Стаж оказался больше Возраста? Найдем эти строчки.

In [40]:
#глянем на всю таблицу
age_cracked_data = data[(data['days_employed'] /365) > data['dob_years']]
print(age_cracked_data.head())
print('')

#потом на Тип занятости у них
print('Стаж больше возраста только у: ', age_cracked_data['income_type'].unique())
print('')

#посмотрим на ряд уникальных значений в возрасте, неужели наших рук дело
print('Ряд уникальных возрастов у этих строк: ', age_cracked_data['dob_years'].unique())
print('')

#также посчитаем, сколько строк с искусственным средним в стаже
print('Из них количество строк со вставленным средним: ', len(age_cracked_data[age_cracked_data['days_employed'] == 15208.478801869218]))

#посмотрим на безработных
print(data[data['income_type'] == 'безработный'])

      children  days_employed  dob_years education  education_id  \
157          0   14517.251167       38.0   среднее             1   
751          0   16281.477669       41.0   среднее             1   
776          0   15222.356680       38.0   среднее             1   
1242         0   13948.510826       22.0   Среднее             1   
1383         0   14741.783820       37.0   среднее             1   

              family_status  family_status_id gender income_type  debt  \
157         женат / замужем                 0      F   пенсионер     1   
751         женат / замужем                 0      M   пенсионер     0   
776         женат / замужем                 0      F   пенсионер     0   
1242  Не женат / не замужем                 4      F   пенсионер     0   
1383         вдовец / вдова                 2      F   пенсионер     0   

       total_income                          purpose  
157   113560.650035             сделка с автомобилем  
751   151898.693438  операции со сво

И тут мы внезапно обнаруживаем, что у 61 пенсионера в выборке стаж больше чем возраст. Причем это не погрешность при заполнении нами нулей средним возрастом, эти данные уже были в исходнике (кстати всего 8 из них с нашим средним значением в столбце Стаж). 

Как так получилось - непонятно. Может быть некорректно посчитался стаж из трудовой книжки. С возрастом ошибиться сложно, но вероятнее всего  операторы некорректно проставили Тип занятости. Здесь только гадать, но в принципе 61 строчка не так сильно повлияют на всю выборку, это всего лишь 0,3% от нее - будем считать допустимой погрешностью.

**Вывод**

Пропусков в исходнике было не так много, и мы не удалили ни одну строчку, а заменили их на медианные/средние значения по категориям, чтобы максимально сохранить репрезентативность. Однако, чтобы найти эти медианы/средние, потребовалось еще больше времени, т.к. данные изначально были некорректными.


### Замена типа данных

In [41]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null float64
dob_years           21525 non-null float64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 2.0+ MB


**Вывод**

К типам данных замечаний нет. Все численные объекты имеют корректный тип int или float, проблем с их использованием быть не должно. Поменялся тип данных у столбца Возраст с int на float, т.к. мы выше заменили там нули на средние значения.

### Обработка дубликатов

Здесь цель убрать дубликаты в целых строках, а также посмотреть на занчения в столбце Образование.
Сначала посмотрим в столбец Образование. 

In [42]:
print('Значения в столбце Образование:', data['education'].unique())

Значения в столбце Образование: ['высшее' 'среднее' 'Среднее' 'СРЕДНЕЕ' 'ВЫСШЕЕ' 'неоконченное высшее'
 'начальное' 'Высшее' 'НЕОКОНЧЕННОЕ ВЫСШЕЕ' 'Неоконченное высшее'
 'НАЧАЛЬНОЕ' 'Начальное' 'Ученая степень' 'УЧЕНАЯ СТЕПЕНЬ'
 'ученая степень']


Со столбцом Образование все понятно: беда с регистром. Видимо операторы заполняют это поле как хотят. Хорошо что без опечаток. Возьмем за стандарт написание (ученая степень, высшее, неоконченное высшее, среднее, начальное) и применим метод str.lower().

In [43]:
data['education'] = data['education'].str.lower()
print('Значения в столбце Образование:', data['education'].unique())

Значения в столбце Образование: ['высшее' 'среднее' 'неоконченное высшее' 'начальное' 'ученая степень']


И посчитаем общее количество дубликатов.

In [44]:
print('Всего дубликатов: ', data.duplicated().sum())

Всего дубликатов:  71


Всего 71 строчка. Убираем.

In [45]:
data = data.drop_duplicates().reset_index (drop = True)
print ('Осталось дубликатов: ', data.duplicated().sum())

Осталось дубликатов:  0


**Вывод**

Из 21 тысячи строк задублировалось всего 71 строчка (меньше 3 процентов). Показатель допустимый, можно как вариант списать это на результат замены пропусков, а также на вероятность того, что один и тот же клиент оформил более 1 кредитного договора с одинаковой целью (например, в случае с ипотекой это необходимо) и одинаково хорошо/плохо его обслуживал. Но уникальных айди кредитных договоров у нас нет, поэтому это всего лишь догадка.

Убрали дубликаты и причесали регистр в столбце Образование. Раз-два, и готово, всегда бы так.


### Лемматизация

Лемматизируем данные в столбце Цель кредита

In [46]:
from pymystem3 import Mystem
m = Mystem() 

# выведем столбец со значениями цели кредита в отдельную таблицу и сразу склеим его
text_purpose = ' '.join(data['purpose'])

# далее сделаем подсчет методом counter()
from collections import Counter
lemmas_purpose = Counter(m.lemmatize(text_purpose))

#и для наглядности выведем полученное в датафрейм
lemmas_purpose_table = pd.DataFrame.from_dict(lemmas_purpose, orient='index').reset_index()

lemmas_purpose_table.sort_values(by = 0, ascending = False)

,index,0
1,,55023
15,недвижимость,6351
0,покупка,5897
2,жилье,4460
4,автомобиль,4306
6,образование,4013
10,с,2918
9,операция,2604
8,свадьба,2324
21,свой,2230


**Вывод**

Прослеживаются следующие цели кредита: недвижимость, автомобиль, образование, свадьба и ремонт. Диверсифицировать недвижимость на жилую и нежилую не будем, т.к. это одинаковый вид кредита - ипотечный, то есть покупка недвижимости с последующей передачей в залог банку. И судя по выгрузке, клиенты рассматривались только как физические лица и их ещемесячные доходы, на основе которых вероятно строилась модель платежеспособности, и банк не добавлял в эту модель например будущие потоки от использования ком. недвижимости, поэтому разбивка на жилую и нежилую лишняя и бессмысленная. 

### Категоризация данных

Для начала прокатегоизируем данные в столбце Цель кредита: нужно чтобы вместо 38 значений в исходнике мы могли оперировать 5ю. Сначала заменим строчки "ремонт жилью" на просто "ремонт", т.к. здесь будут конфликтовать леммы Жилье и Ремонт, а нам надо, чтобы эти строчки ушли в категорию Ремонт.

In [47]:
#заменим Ремонт жилью на Ремонт
data['purpose'] = data['purpose'].replace('ремонт жилью', 'ремонт')

#напишем функцию для лемматизации столбца
def purpose_category(row):
    purpose = row['purpose']
    lemmas = m.lemmatize(purpose)
    if 'ремонт' in lemmas:
        return 'ремонт'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'свадьба' in lemmas:
        return 'свадьба'
    if 'образование' in lemmas:
        return 'образование'
    if 'недвижимость' in lemmas or 'жилье' in lemmas:
        return 'недвижимость'
    return 'прочее'

#и вставим ее в нашу базу и сразу посмотрим результат
data['purpose_group'] = data.apply(purpose_category, axis = 1)
data['purpose_group'].value_counts()
    

недвижимость    10204
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: purpose_group, dtype: int64

Теперь необходимо категоризировать столбец Доходы. Воспользуемся методом qcut для выделения 5 равных групп доходов и дадим им значения на понятном каждому языке, например буквенные значения, как оценки в школе.

In [48]:
#добавляем столбец с категорией дохода 
income_group_labels = ['E', 'D', 'C', 'B', 'A']

data['total_income_group'] = pd.qcut(data['total_income'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=income_group_labels)

#выведем таблицу значений групп:
data['total_income_group_num'] = pd.qcut(data['total_income'], q=5)
print(data['total_income_group_num'].value_counts())

#смотрим итог деления по этим персентелей:
print(data['total_income_group'].value_counts())
data.head()

#и сведем словарь
dict_income = [
    ['A', '(214618.469, 2265604.029]'],
    ['B','(161335.699, 214618.469]'],
    ['C','(132135.053, 161335.699]'],
    ['D','(98538.268, 132135.053]'],
    ['E','(20667.263, 98538.268]'],
] 

entries = ['group', 'income']

total_income_group_num = pd.DataFrame(data = dict_income , columns = entries)
print(total_income_group_num)

(20667.263, 94836.83]        3871
(223107.554, 2265604.029]    3870
(164660.939, 223107.554]     3870
(128279.674, 164660.939]     3870
(94836.83, 128279.674]       3870
Name: total_income_group_num, dtype: int64
E    3871
A    3870
B    3870
C    3870
D    3870
Name: total_income_group, dtype: int64
  group                     income
0     A  (214618.469, 2265604.029]
1     B   (161335.699, 214618.469]
2     C   (132135.053, 161335.699]
3     D    (98538.268, 132135.053]
4     E     (20667.263, 98538.268]


<div class="alert alert-success" style="border-radius: 15px; box-shadow: 4px 4px 4px; border: 1px solid ">
<b> Комментарий ревьюера</b>
    
Отличная идея :)</div>

Соберем вместе все словари для наглядности

In [49]:
education_dict = data[['education', 'education_id']]
education_dict = education_dict.drop_duplicates().reset_index(drop=True)
print(education_dict) 
print(' ') 
family_status_dict = data[['family_status', 'family_status_id']]
family_status_dict = family_status_dict.drop_duplicates().reset_index(drop=True)
print(family_status_dict) 
print(' ') 
print(total_income_group_num)
print(' ') 
print(data['purpose_group'].value_counts())

             education  education_id
0               высшее             0
1              среднее             1
2  неоконченное высшее             2
3            начальное             3
4       ученая степень             4
 
           family_status  family_status_id
0        женат / замужем                 0
1       гражданский брак                 1
2         вдовец / вдова                 2
3              в разводе                 3
4  Не женат / не замужем                 4
 
  group                     income
0     A  (214618.469, 2265604.029]
1     B   (161335.699, 214618.469]
2     C   (132135.053, 161335.699]
3     D    (98538.268, 132135.053]
4     E     (20667.263, 98538.268]
 
недвижимость    10204
автомобиль       4306
образование      4013
свадьба          2324
ремонт            607
Name: purpose_group, dtype: int64


**Вывод**

Проведена категоризация по цели кредита по максимально простым категориям для физ.лиц: недвижимость, автомобиль, образование, свадьба и ремонт.
Категории дохода получили путем равного распределения всех на 5 частей.
И на будущее совет банковским разработчикам интерфейса  - максимально упростить ввод данных. Например, выбор только из выпадающего списка или ограниченный формат ввода, в общем, нужно минимизировать ручной ввод для чистоты данных и исключения влияния человеческого фактора.

В изначальной таблице есть словари по типу Образования и Семейного положения.
Для удобства анализа были добавлены два словаря: для Целей кредита и Дохода. 
Цели кредитования были укрупнены до 5 принципиально разных категорий, а Доход, точнее его распределение по выборке было поделено на 5 равных персентелей, чтобы визуально выделить заемщиков с низким, средним и высоким доходом.

## Шаг 3. Ответить на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

Здесь просто вычислим долю просрочки в категориях "есть дети" и "нет детей"

In [50]:
#посчитаем сначала среднюю температуру по больнице
debt_rate = len(data [data['debt'] > 0]) / len(data)

#делим количество заемщиков с детьми и просрочкой на общее количетсво заемщиков с детьми
children_have_rate = len(data [ (data['debt'] > 0) & (data['children'] > 0) ])  / len(data[data['children'] > 0])

#делим количество заемщиков без детей и просрочкой на общее количетсво заемщиков без детей
children_not_have_rate = len(data [ (data['debt'] > 0) & (data['children'] == 0) ])  / len(data[data['children'] == 0])

#оформляем вывод
print ('Общая доля просрочки среди всех заемщиков: {:.1%}'.format(debt_rate))
print ('Доля просрочки среди заемщиков с детьми: {:.1%}'.format(children_have_rate))
print ('Доля просрочки среди заемщиков без детей: {:.1%}'.format(children_not_have_rate))

#и сделаем  таблицу, чтобы увидеть разбивку по количеству детей
children_rate = data.groupby ('children').agg({'debt': ['mean', 'count']})
children_rate.columns = ['доля просрочки', 'всего заемщиков']
children_rate = children_rate.sort_values (by = 'доля просрочки', ascending = False)
children_rate.style.format({'доля просрочки':'{:.2%}'})

Общая доля просрочки среди всех заемщиков: 8.1%
Доля просрочки среди заемщиков с детьми: 9.2%
Доля просрочки среди заемщиков без детей: 7.5%


,доля просрочки,всего заемщиков
children,,
4,9.76%,41
2,9.49%,2128
1,9.17%,4855
3,8.18%,330
0,7.54%,14091
5,0.00%,9


**Вывод**

Разница между долей просроченных кредитов есть, но она минимальная - 1,7 процентных пункта или на 22,7% больше.
С первого взгляда кажется, что наличие иждивенцев в семье приводит к доп. тратам и тем самым повышается риск невыплаты кредита, однако цифры полностью это не подтверждают: грубо говоря из 100 человек без детей с просрочкой 7, а с детьми 9 человек. Здесь веских оснований пересматривать кредитную политику в отношении заемщиков с детьми нет, выход на просрочку статистически неизбежен. И формирование повышенного резерва на возможные потери по ссудам в этой категории не требуется: банк точно заработает на 91 кредите из 100 в этой категории при общем показателе своевременного погашения 91,9%.

Если дополнительно посмотреть разбивку именно по количеству детей, то там четкой взаимосвязи вообще нет: заемщики с 3 детьми допускают просрочку в среднем в 8,18% случаев, с двумя детьми - 9,49%, а с 1 ребенком - 9,17% что уже вроде как нелогично и гипотеза "чем больше детей, тем больше вероятность просрочки" не подтверждается в принципе.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [51]:
#сделаем таблицу

marrige_rate = data.groupby ('family_status').agg({'debt': ['mean', 'count']})
marrige_rate.columns = ['доля просрочки', 'всего заемщиков']
marrige_rate = marrige_rate.sort_values (by = 'доля просрочки', ascending = False)
marrige_rate.style.format({'доля просрочки':'{:.2%}'})

,доля просрочки,всего заемщиков
family_status,,
Не женат / не замужем,9.75%,2810
гражданский брак,9.35%,4151
женат / замужем,7.55%,12339
в разводе,7.11%,1195
вдовец / вдова,6.57%,959


**Вывод**

Самая большая доля просрочки у групп заемщиков в статусе "Не женат / не замужем" и "гражданский брак", то есть у тех кто не состоит/не состоял в браке. Хотя стоит отметить, что разница перед следующей категорией состовляет около 2 процентных пункта, что по сути не является чем-то из ряда вон выходящим.
И можно сделать гипотезу о том, что люди, которые оформили свои отношения официально (или имели в этом опыт), хоть и не на много, но имеют более высокую платежную дисциплину и стабильность. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [52]:
#освежим данные по категориям доходов
print(total_income_group_num)

#сделаем таблицу на подобие той, которую делали выше

income_rate = data.groupby ('total_income_group').agg({'debt': ['mean', 'count']})
income_rate.columns = ['доля просрочки', 'всего заемщиков']
income_rate = income_rate.sort_values (by = 'доля просрочки', ascending = False)
income_rate.style.format({'доля просрочки':'{:.2%}'})



  group                     income
0     A  (214618.469, 2265604.029]
1     B   (161335.699, 214618.469]
2     C   (132135.053, 161335.699]
3     D    (98538.268, 132135.053]
4     E     (20667.263, 98538.268]


,доля просрочки,всего заемщиков
total_income_group,,
C,8.66%,3870
B,8.50%,3870
D,8.37%,3870
E,7.98%,3871
A,7.08%,3870


**Вывод**

- Как разные цели кредита влияют на его возврат в срок?

In [53]:
#выведем таблицу с необходимыми данными
purpose_rate = data.groupby ('purpose_group').agg({'debt': ['mean', 'count']})
purpose_rate.columns = ['доля просрочки', 'всего заемщиков']
purpose_rate = purpose_rate.sort_values (by = 'доля просрочки', ascending = False)
purpose_rate.style.format({'доля просрочки':'{:.2%}'})

,доля просрочки,всего заемщиков
purpose_group,,
автомобиль,9.36%,4306
образование,9.22%,4013
свадьба,8.00%,2324
недвижимость,7.32%,10204
ремонт,5.77%,607


**Вывод**

В топе просрочки мы можем наблюдать автокредит и образовательный кредит, а в боттоме ипотеку и с относительно большим отрывом ремонт. Аналогично предыдущим срезам, здесь есть зависимость, но она минимальна.

## Шаг 4. Общий вывод

Исходные данные представлены с большим количеством ошибок и нестыковок. Это может быть связано например с тем, что данные объединяются из нескольких разных учетных систем, и поэтому появляются такие огрехи, как отрицатеьный стаж или стаж в часах. А также на исходник повлиял человеческий фактор (возраст = 0, разный регистр в образовании, разношерстный набор целей кредитования). Здесь можно дать рекомендацию максимально упростить формы ввода для операторов: выбор из списка вместо рукописного ввода, автоматическая проверка данных при вводе.

Что касается выявления зависимостей наличия просрочки по разным категориям заемщиков, то здесь выделить однозначные направления для изменения кредитной политики банка нельзя. И все же в рамках поручения "обратить внимание" можно выделить заемщиков с самой высокой долей просрочки: это люди в семейном статусе "не женат/ незамужем" (9,75%) и "в гражданском браке"(9,35%), а также с целью кредита на Авто (9.36%) и Образование (9.22%). А относительно хорошими заемщиками являются вдовы/вдовцы (6.57%) и с целевым использованием кредита Ремонт (5.77%). 

Если говорить в целом о качестве кредитного портфеля, то средняя просрочка в нем 8,1%. По данным статистики ЦБ за 2020 год средняя по РФ доля просроченных кредитов составила 4,38% (https://ria.ru/20210301/kredit-1599357511.html). Поэтому здесь кредитому отделу рекомендация оценить разницу со своим регионом присутствия (там разброс от 1,66 до 7,8 в зависимости от региона) и отталкиваясь от этого разработать направления движения для усовершенствования своей системы кредитного скоринга.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- []  есть общий вывод.